In [19]:
# import os

# # List files in the input directory
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [20]:
# !pwd
# import sys
# sys.path.append('/kaggle/working/packages')  # Replace with the actual path
# !echo $PYTHONPATH

**To delete a specific folder of kaggle**

In [21]:
# # Clear output folder
# import os

# def remove_folder_contents(folder):
#     for the_file in os.listdir(folder):
#         file_path = os.path.join(folder, the_file)
#         try:
#             if os.path.isfile(file_path):
#                 os.unlink(file_path)
#             elif os.path.isdir(file_path):
#                 remove_folder_contents(file_path)
#                 os.rmdir(file_path)
#         except Exception as e:
#             print(e)

# folder_path = '/kaggle/working/packages'
# remove_folder_contents(folder_path)
# os.rmdir(folder_path)

In [22]:
# !pip install --upgrade accelerate peft bitsandbytes transformers pdfplumber datasets==2.16.1 trl==0.7.9 --target=/kaggle/working/packages
%pip install --upgrade accelerate peft transformers pdfplumber datasets==2.16.1 trl
%pip install bitsandbytes --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui



Looking in indexes: https://pypi.org/simple, https://jllllll.github.io/bitsandbytes-windows-webui
Note: you may need to restart the kernel to use updated packages.


**To resolve some issues while installing packages**

In [23]:
# import pkg_resources

# def resolve_conflicts():
#     """
#     Resolves dependency conflicts by creating a new environment and installing compatible versions.
#     """

#     # Create a new virtual environment (replace "myvenv" with your desired name)
#     !virtualenv myvenv
#     !source myvenv/bin/activate

#     # Install compatible versions of conflicting packages
#     required_versions = {
#         "cupy-cuda11x": ">=12.0.0",
#         "dill": "<0.3.2,>=0.3.1.1",
#         "numpy": "<1.25.0,>=1.14.3",
#         "pyarrow": "<10.0.0,>=3.0.0",
#         "jupyter-server": "~=1.16",
#         "jupyterlab": "~=3.4",
#         "urllib3": "<2.1,>=1.25.4",  # Ensure python_version >= "3.10"
#         "pandas": "<1.6.0dev0,>=1.3",
#         "protobuf": "<5,>=4.21",
#         "dask": "==2023.7.1",
#         "distributed": "==2023.7.1",
#         "fsspec": "==2023.6.0",
#         "urllib3": "<2.0",
#         "google-api-core[grpc]": "<2.0.0dev,>=1.22.2",
#         "packaging": "<22.0dev,>=14.3",
#         "grpcio": "<2.0dev,>=1.51.3",
#         "jupyter-lsp": ">=2.0.0",
#         "google-cloud-storage": "<3,>=2.2.1",
#         "shapely": ">=2.0.1",
#         "numpy": "<1.25,>=1.21",
#         "cryptography": "<42,>=38.0.0",
#         "numpy": "<1.22.2,>=1.15.0",
#         "scipy": "<1.8.0,>=1.7.3",
#         "fsspec": "==2023.12.2",
#         "typing-extensions": "<4.6.0,>=3.6.6",
#         "platformdirs": "<4,>=2.4",
#         "numpy": "<1.24,>=1.16.0",
#         "pandas": "!=1.4.0,<2.1,>1.1",
#         "scipy": "<1.12,>=1.4.1",
#     }

#     for package, version_spec in required_versions.items():
#         !pip install "{package} {version_spec}"

#     # Verify that conflicts are resolved
#     working_set = pkg_resources.WorkingSet()
#     if not working_set.require(required_versions):
#         raise RuntimeError("Dependency conflicts could not be resolved.")

#     print("Dependency conflicts resolved!")

# if __name__ == "__main__":
#     resolve_conflicts()


**Importing necessary packages**

In [24]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

**Model: https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v0.3?text=My+name+is+Clara+and+I+am**

In [25]:
# Model from Hugging Face hub
base_model = "PY007/TinyLlama-1.1B-Chat-v0.3"

# Fine-tuned model
new_model = "TinyLlama-chat-medaid-model"

**Reading the PDF and storing it as a text file**

In [26]:
# import pdfplumber
# from datasets import load_dataset

# # Path to your PDF file
# pdf_path = "/kaggle/input/doctor-patient-conversation/combined_conversations_with_scenarios.pdf"

# # Extract text from PDF using pdfplumber
# with pdfplumber.open(pdf_path) as pdf:
#     text = ""
#     for page in pdf.pages:
#         text += page.extract_text()

# # Save the extracted text to a file in kaggle output
# output_path = "/kaggle/working/output_file_dataset.txt"  # Use the /content directory in Colab
# with open(output_path, "w", encoding="utf-8") as output_file:
#     output_file.write(text)

# # Now you can load the dataset using the load_dataset function
# dataset = load_dataset("text", data_files=output_path, split="train")

# READING THE NEW CONVERSATION DATASET WITH ALL THE TEXT FILES (doctor-patient-conversation-large)

# Directory containing your text files
text_files_directory = r"G:\LLM-Model-MedAid-Thesis\Model-3\Dataset"

# List to store individual conversation texts
conversation_texts = []

# Loop through each text file in the directory
for filename in os.listdir(text_files_directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(text_files_directory, filename)
        with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
            # Use errors="ignore" to skip characters that cannot be decoded
            conversation_texts.append(file.read())

# Save the combined text to a file
output_path = r"G:\LLM-Model-MedAid-Thesis\Model-3\combined_conversations.txt"
with open(output_path, "w", encoding="utf-8") as output_file:
    for text in conversation_texts:
        output_file.write(text + "\n\n")

dataset = load_dataset("text", data_files=output_path, split="train")


Generating train split: 53861 examples [00:00, 285108.84 examples/s]


In [27]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 53861
})


**This code has a configuration for quantization using the *BitsAndBytesConfig class* from the *trl library*. Quantization is a technique used in deep learning to reduce the memory and computational requirements of neural networks.**

In [28]:
# compute_dtype = getattr(torch, "float16")

# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=False,
# )

import torch
from torch import nn
from torch.quantization import QuantStub, DeQuantStub, quantize_dynamic

# Define your model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.quant = QuantStub()
        self.dequant = DeQuantStub()
        self.linear = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.quant(x)
        x = self.linear(x)
        x = self.dequant(x)
        return x

# Instantiate the model
model = MyModel()


**This code is using the AutoModelForCausalLM class from the transformers library to instantiate a pre-trained causal language model with specific configurations.**

In [29]:
# %python -m bitsandbytes
# %pip install --upgrade bitsandbytes
# import torch
# print(torch.cuda.is_available())
from torch.utils.cpp_extension import CUDA_HOME
print(CUDA_HOME) # by default it is set to /usr/local/cuda/

C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1


In [30]:
# specify that TensorFlow performs computations using the CPU
import os
os.environ['TF_ENABLE_MLIR_OPTIMIZATIONS'] = '1'

In [31]:
import torch
print("Cuda ", torch.cuda.is_available())

# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=quant_config,
#     # device_map={"": 0}
#      device_map={"": "cpu"}
# )

device = torch.device('cpu')
model.to(device)
model = AutoModelForCausalLM.from_pretrained(base_model).to(device)


model.config.use_cache = False
model.config.pretraining_tp = 1


Cuda  True


In [32]:

# Quantize the model dynamically
quantized_model = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

**This code is using the AutoTokenizer class from the transformers library to instantiate a tokenizer for a pre-trained language model.**

In [33]:
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

# Create the tokenizer on the chosen device
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True, device=device)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**This code is defining a configuration for the LoRA (Local Reparameterization with Attention) model using the LoraConfig class, which appears to be part of the peft library.**

In [34]:
# Create a LoraConfig instance
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

**This code is defining a set of training parameters using the TrainingArguments class, which is often used in the transformers library for configuring training settings.**

In [35]:
# training_params = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs=1,
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=1,
#     optim="paged_adamw_32bit",
#     save_steps=25,
#     logging_steps=25,
#     learning_rate=2e-4,
#     weight_decay=0.001,
#     fp16=False,
#     bf16=False,
#     max_grad_norm=0.3,
#     max_steps=-1,
#     warmup_ratio=0.03,
#     group_by_length=True,
#     lr_scheduler_type="constant",
#     report_to="tensorboard"
# )

# Specify the output directory and other training parameters
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

**This code is creating an instance of the SFTTrainer class, presumably from the trl library, to facilitate the training of a model using the specified configuration.**

**SFTTrainer instance is configured with the** 
* model, 
* training dataset, 
* Peft configuration, 
* tokenizer,
* training arguments. 

**The specific behavior and training process are determined by the SFTTrainer implementation in the trl library, and the configured parameters influence aspects such as optimization, learning rate, and model architecture during training.**

In [36]:
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_params,
#     dataset_text_field="text",
#     max_seq_length=None,
#     tokenizer=tokenizer,
#     args=training_params,
#     packing=False,
# )

# Create the trainer
trainer = SFTTrainer(
    model=quantized_model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C'), WindowsPath('/Users/User/anaconda3/envs/tinyLlamaMedAid/lib')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('file'), WindowsPath('/Users/User/anaconda3/envs/tinyLlamaMedAid/etc/xml/catalog'), WindowsPath('/C')}
The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('/matplotlib_inline.backend_inline')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SET

RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
trainer.train()

**The code you provided is saving the trained model and its associated tokenizer to a specified directory using the save_pretrained method.**

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

**This code creates a simple conversational loop that simulates a doctor assistant interaction. It uses a fine-tuned language model (assumed to be a text generation model) and a tokenizer to generate responses based on user input. The conversation is logged in a text file, and the loop continues until the user provides an exit signal.**

In [ ]:
# logging.set_verbosity(logging.CRITICAL)

# Initial prompt
prompt = "Be a doctor assistant. And keep questioning one by one to extract symptoms and history of the patient. Don't give advice or ask anything else. Just extract symptoms or history by questioning one question at a time."

# Create a pipeline for text generation using the fine-tuned model and tokenizer
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

# File to save the conversation
output_file_path = "G:\LLM-Model-MedAid-Thesis\Model-3\conversation_log.txt"  # Update 'your_folder' with the desired folder in your Drive

# Function to ask a question and get the user's response
def ask_question_and_log(prompt, user_response, file_path):
    # Ask the question
    result = pipe(f"<s>[Prompt] {prompt} \n [User response] {user_response} \n [/Response]")

    # Get the generated text (question)
    generated_text = result[0]['generated_text']

    # Print and save the generated text (question)
    print(generated_text)
    with open(file_path, "a", encoding="utf-8") as output_file:
        output_file.write(f"[Model] {generated_text}\n\n")

    # Simulate user answering the question
    user_response = input("Your response: ")  # User provides input

    return user_response

# Initial user response
user_response = "I have chest pain."

# Ask a question based on the user's response and log the conversation
with open(output_file_path, "a", encoding="utf-8") as output_file:
    output_file.write("\nConversation started.\n\n")

# Loop to continue the conversation
while True:
    user_response = ask_question_and_log(prompt, user_response, output_file_path)

    # Check for an exit condition (e.g., user response indicating the end of the conversation)
    if "exit" in user_response.lower():
        print("Ending the conversation.")

        # Save the conversation to a file
        with open(output_file_path, "a", encoding="utf-8") as output_file:
            output_file.write("\nConversation ended by user. ---------------- \n\n")

        break
